In [1]:
import numpy as np
import pandas as pd 
import plotly.express as px

In [2]:
SEED = 42

## Carga de datos

In [3]:
df = pd.read_csv('/kaggle/input/car-price-dataset/car_price_dataset.csv')
df.sample(5, random_state=SEED)

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
6252,BMW,3 Series,2005,1.2,Diesel,Manual,257760,5,5,2000
4684,Volkswagen,Golf,2013,2.9,Hybrid,Automatic,111790,5,3,11164
1731,Toyota,RAV4,2011,5.0,Hybrid,Automatic,13473,5,4,14630
4742,Mercedes,E-Class,2007,3.8,Diesel,Manual,133298,4,5,7334
4521,Honda,Civic,2012,1.3,Petrol,Automatic,18611,3,5,10127


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         10000 non-null  object 
 1   Model         10000 non-null  object 
 2   Year          10000 non-null  int64  
 3   Engine_Size   10000 non-null  float64
 4   Fuel_Type     10000 non-null  object 
 5   Transmission  10000 non-null  object 
 6   Mileage       10000 non-null  int64  
 7   Doors         10000 non-null  int64  
 8   Owner_Count   10000 non-null  int64  
 9   Price         10000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 781.4+ KB


In [5]:
df.describe()

,Year,Engine_Size,Mileage,Doors,Owner_Count,Price
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,2011.543700,3.000560,149239.111800,3.497100,2.991100,8852.96440
std,6.897699,1.149324,86322.348957,1.110097,1.422682,3112.59681
min,2000.000000,1.000000,25.000000,2.000000,1.000000,2000.00000
25%,2006.000000,2.000000,74649.250000,3.000000,2.000000,6646.00000
50%,2012.000000,3.000000,149587.000000,3.000000,3.000000,8858.50000
75%,2017.000000,4.000000,223577.500000,4.000000,4.000000,11086.50000
max,2023.000000,5.000000,299947.000000,5.000000,5.000000,18301.00000


- **Year** -> año de producción. El registro va desde el año 2000 hasta el 2023
- **Engine_Size** -> tamaño del motor en Litros. Tenemos que ver si tratarla como variable numerica o categórica, ya que no es un valor continuo como tal.
- **Mileage** -> millas/cuenta kilometros. La desviación estándar es muy alta y fijandonos en los valores max y min, confirmamos la disparidad entre coches muy usados y apenas sacados del garaje. Sin duda, sera una variable clave para determinar el precio.
- **Doors** -> número de puertas. De 1 hasta 5 (el maletero cuenta como puerta). Vemos que la mitad de los registros (q50%) tiene 3 puertas, lo que indica que son modelos de coches compactos. Podemos tratarla como categorica *ordinal*
- **Owner_Count** -> número de propietarios. Desde 1 hasta 5, siendo 3 la media (habría que ver la moda). Podemos tratarla como categorica *ordinal*
- **Price** -> es variable objetivo, no se indica tipo de moneda, así por simpleza lo denominaremos como "unidades monetarias"

In [6]:
df['Engine_Size'].unique()

array([4.2, 2. , 2.6, 2.7, 3.4, 4.7, 3.1, 1.3, 4.5, 4.8, 2.3, 2.4, 3.2,
       2.9, 1.5, 1.8, 4.6, 2.8, 3.9, 3.3, 1.9, 3.5, 1.4, 2.2, 1. , 1.2,
       4.9, 3.6, 3.8, 4. , 1.7, 1.6, 4.1, 1.1, 2.5, 3. , 5. , 2.1, 3.7,
       4.3, 4.4])

Vemos que no son valores exactos (coma flotante) por lo que podríamos discretizar la variable en el procesamiento y tratarla como categorica.

**NOTA** Al reagrupar podemos los modelos lineales pueden discernir mejor

In [7]:
df['Owner_Count'].value_counts()

Owner_Count
5    2036
1    2036
2    2020
3    1977
4    1931
Name: count, dtype: int64

Tiene una distribución muy equilibrada, siendo lo más común los "extremos": o bien solo 1 propietario o bien 5

In [8]:
df.describe(include="object")

,Brand,Model,Fuel_Type,Transmission
count,10000,10000,10000,10000
unique,10,30,4,3
top,Ford,Accord,Electric,Manual
freq,1048,365,2625,3372


In [9]:
df['Brand'].unique()

array(['Kia', 'Chevrolet', 'Mercedes', 'Audi', 'Volkswagen', 'Toyota',
       'Honda', 'BMW', 'Hyundai', 'Ford'], dtype=object)

In [10]:
df['Fuel_Type'].unique()

array(['Diesel', 'Hybrid', 'Electric', 'Petrol'], dtype=object)

In [11]:
df['Transmission'].unique()

array(['Manual', 'Automatic', 'Semi-Automatic'], dtype=object)

De un primer vistazo vemos los valores únicos para cada variable, con las visualizaciones veremos mejor sus distribuciones. También podemos usar el método `value_counts` si no queremos crear gráficas. 

**NOTA** Estas variables las podemos tratar como `category` por su naturaleza.

In [12]:
cat_cols = df.select_dtypes(include="object").columns.to_list() + ["Doors", "Owner_Count"]
df[cat_cols] = df[cat_cols].astype('category')

In [13]:
df.isna().sum() / df.shape[0]

Brand           0.0
Model           0.0
Year            0.0
Engine_Size     0.0
Fuel_Type       0.0
Transmission    0.0
Mileage         0.0
Doors           0.0
Owner_Count     0.0
Price           0.0
dtype: float64

In [14]:
df.duplicated().sum()

0

No hay valores perdidos ni tampoco duplicados, por lo que no será necesario realizar utilizar técnicas complejas en el procesamiento de los datos

## Visualización de los datos

In [15]:
num_features = df.select_dtypes(include=np.number).columns.to_list()
cat_features = df.select_dtypes(include="category").columns.to_list()
print("Variables numéricas: ", num_features)
print("Variables categóricas: ", cat_features)

Variables numéricas:  ['Year', 'Engine_Size', 'Mileage', 'Price']
Variables categóricas:  ['Brand', 'Model', 'Fuel_Type', 'Transmission', 'Doors', 'Owner_Count']


In [16]:
for cat in cat_features:
    fig = px.histogram(df, x=cat)
    fig.update_layout(bargap=0.2)
    fig.show()

Parece que las diferentes categorías dentro de cada una de las distintas variables categóricas se mantienen distribuidas uniformemente.
Donde se puede notar un ligero desajuste es en los modelos de coches (`model`), pero no es nada significativo

In [17]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [18]:
for num in num_features:
    
    # Create a figure with 1 row, 2 columns
    fig = make_subplots(rows=1, cols=2, subplot_titles=[f"Boxplot of {num}", f"Histogram of {num}"])
    fig.add_trace(
        go.Box(
            y=df[num],
            name=f"{num}",
            boxpoints="outliers",
            jitter=0.5,
            pointpos=-1.5,
            showlegend=False
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Histogram(
            x=df[num],
            showlegend=False
        ),
        row=1, col=2
    )

    fig.show()

- El precio sigue una distribución ligeramente inclinada hacia la izquierda. Si tomamos como centro de la "campana" el valor de 10K, vemos que la mayoría de coches son tasados por debajo de este valor. Mientras que los que están por encima son menos comunes, de ahí los *outliers* en el diagram de cajas. **NOTA** Aparece en el valor 0 algo extraño en el histograma, dado que el valor minimo visto anteriormente son 2K por coche.
- El resto de variables numéricas se comportan de una manera uniforme y ademas tampoco presentan valores atípicos

Hasta ahora hemos visto como se comporta cada variable por separado, veamos ahora como se relacionan entre si

In [19]:
fig = px.imshow(df[num_features].corr().round(2), text_auto=True)
fig.show()

- El año de producción (`Year`) se correlaciona directamente con el precio. "Más reciente, más nuevo, mayor precio"
- El kilometraje (`Mileage`), pasa todo lo contrario. "A más millas/kms, más uso, menor es el precio.

**NOTA** Como todavía no hemos discretizado el `engine_size` la correlación es débil, pero esperamos a que cuanto más grande es el motor, mayor sea la tasación del coche

Veamos ahora que pasa con el precio y las variables categóricas

In [20]:
df.groupby(by=['Fuel_Type'], observed=True).agg(mean_price=('Price', 'mean')).sort_values(by="mean_price", ascending=False).reset_index()

,Fuel_Type,mean_price
0,Electric,10032.220190
1,Hybrid,9113.030167
2,Diesel,8117.336385
3,Petrol,8070.561826


Como era de esperar, los coches eléctricos encabezan el ranking de más caros y siendo los de gasolina (petrol) los más baratos con una diferencia de 2K unidades monetarias de media

In [21]:
df_query = (
    df
    .groupby(by=['Brand'], observed=True)
    .agg(mean_price=('Price', 'mean'), std_price=('Price', 'std'))
    .sort_values(by="mean_price", ascending=False)
    .reset_index()
)

In [22]:
fig = px.bar(df_query, x='Brand', y='mean_price', color="mean_price")
fig.show()

Las marcas alemanas son las que encabezan el ranking, sin embargo todas las marcas suelen rondar un mismo rango de precios

In [23]:
(
    df
    .groupby(by=['Brand', 'Model'], observed=True)
    .agg(min_price=('Price', 'min'), max_price=('Price', 'max'), mean_price=('Price', 'mean'), std_price=('Price','std'))
    .sort_values(by='mean_price', ascending=False)
    .head(10)
    .reset_index()
)

,Brand,Model,min_price,max_price,mean_price,std_price
0,Chevrolet,Equinox,2000,17054,9156.320635,2955.802865
1,Volkswagen,Tiguan,2000,16334,9098.863636,2846.175629
2,Mercedes,GLA,2000,16709,9049.844444,3244.167651
3,Chevrolet,Impala,2000,17072,9001.115493,3134.030479
4,Toyota,Corolla,2000,18301,8996.618462,3217.468123
5,Kia,Optima,2000,16893,8996.318452,2968.359427
6,Ford,Fiesta,2000,16705,8971.077135,2912.221809
7,Mercedes,E-Class,2000,15871,8967.330218,2935.544847
8,Audi,Q5,2000,16734,8955.257143,3098.917714
9,Mercedes,C-Class,2000,17614,8921.660131,3158.725731


Un mismo modelo se vende por diferentes precios. Esto seguramente dependa de factores como su kilometraje, año de producción o tipo de motor.

- Vemos que los precios no varian demasiado entre los modelos y marcas
- El rango de precios es bastante dispar (el mínimo y máximo están muy alejados, así como también la desviación estándar es un tanto alta)

In [24]:
(
    df
    .groupby(by=['Transmission'], observed=True)
    .agg(mean_price=('Price', 'mean'))
    .sort_values(by="mean_price",ascending=False)
    .reset_index()
)

,Transmission,mean_price
0,Automatic,9938.252939
1,Manual,8363.426157
2,Semi-Automatic,8264.266385


Los coches automáticos son los más caros, dado que la transmisión requieren de más componentes.

In [25]:
(
    df
    .groupby(by=['Doors'], observed=True)
    .agg(mean_price=('Price', 'mean'))
    .sort_values(by="mean_price",ascending=False)
    .reset_index()
)

,Doors,mean_price
0,2,8928.931161
1,5,8900.352606
2,4,8841.587908
3,3,8746.446602


In [26]:
fig = px.violin(df, x="Doors", y="Price", box=True, points='all', color="Doors", title="Precio de Coche por Nº de Puertas")
fig.show()

Sorprende ver que los doches de dos puertas sean los más caros de media, puede atribuirse a que suelen ser modelos más deportivos. 
Aun así la diferencia no es tan grande con los de cinco puertas. Quedando a la cola los compactos de tres y cuatro puertas

In [27]:
df.groupby(by=['Owner_Count'], observed=True).agg(mean_price=('Price', 'mean')).sort_values(by="mean_price").reset_index()

,Owner_Count,mean_price
0,1,8841.374263
1,2,8847.258416
2,3,8847.918563
3,5,8859.742141
4,4,8869.173485


In [28]:
fig = px.box(df, x="Owner_Count", y="Price", color="Owner_Count", title="Car Price by Number of Owners")
fig.show()


No hay diferencias de precio significativas entre los distintos propietarios, lo que indica que factores como el estado y el kilometraje pueden influir más en el precio.

## Procesamiento de los datos